# TP : Spark SQL

Imagine you're analyzing sales data for a retail company. You need to answer questions like: "What is the average order value per customer?" and "Which products are most popular in each region?"

By using Spark SQL to query structured data in a DataFrame, you can efficiently analyze large datasets that would be cumbersome with traditional SQL.

In [1]:
import pandas as pd
import numpy as np

# Generate a fake dataset
data = {
    'transaction_id': np.arange(1, 500001),
    'product_id': np.random.randint(1, 1000, 500000),
    'product_category': np.random.choice(['Electronics', 'Clothing', 'Books'], 500000),
    'region': np.random.choice(['North', 'South', 'East', 'West'], 500000),
    'date': pd.date_range(start="2021-01-01", periods=500000, freq='T').to_numpy(),
    'units_sold': np.random.randint(1, 10, 500000),
    'price_per_unit': np.round(np.random.uniform(5.0, 500.0, 500000), 2)
}
df = pd.DataFrame(data)

<ipython-input-1-6053c13f21be>:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'date': pd.date_range(start="2021-01-01", periods=500000, freq='T').to_numpy(),


TODO: convert to Spark Dataframe

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Sales Analysis").getOrCreate()

In [14]:
sales_df = spark.createDataFrame(df)

TODO: Register DataFrame as a SQL temporary view to be able to write queries


In [15]:
#sales_df.

In [16]:
spark.sql("""
    SELECT region, product_category, SUM(units_sold * price_per_unit) AS total_sales
    FROM sales
    GROUP BY region, product_category
    ORDER BY total_sales DESC
""").show()


+------+----------------+--------------------+
|region|product_category|         total_sales|
+------+----------------+--------------------+
| South|           Books|5.3223514329999894E7|
|  West|           Books| 5.303012518999992E7|
|  East|     Electronics| 5.301111864999989E7|
|  East|        Clothing| 5.299513040999991E7|
| North|           Books| 5.286268056999998E7|
| North|        Clothing|       5.283984331E7|
| South|     Electronics| 5.268548529000001E7|
| South|        Clothing|  5.26344210500001E7|
|  West|        Clothing| 5.231710817999995E7|
|  East|           Books| 5.229177429000023E7|
| North|     Electronics| 5.219497197000006E7|
|  West|     Electronics| 5.214940264999998E7|
+------+----------------+--------------------+



TODO : write and execute query to get the top 5 products by sales

In [17]:
spark.sql("""

""").show()

+----------+-----------------+
|product_id|      total_sales|
+----------+-----------------+
|       976|748420.7199999995|
|       158|744019.4199999999|
|       591|738351.1300000006|
|         3|        726578.54|
|       761|726477.1999999997|
+----------+-----------------+

